**_pySpark Basics: Missing Data_**

_by Jeff Levy (jlevy@urban.org)_

_Last Updated: 21 June 2016, Spark v1.6.1_

_Abstract: In this guide we'll look at how to handle null and missing values in pySpark_

***

We'll begin by verifying the Spark Context and loading the SQL Context necessary to work with a dataframe:

In [1]:
try:
    sc
except NameError:
    raise Exception('Spark context not created.')

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

We'll load some real data from CSV to work with.  It helps to know in advance how the dataset handles missing values - are they an empty string, or something else?  Most CSVs will use empty strings, but we can't compute anything on a column that is mixed strings and numbers.  The `null` object in pySpark is what we want, and we can tell it when we import the data to replace the value our data uses to denote missing data with it.

In [3]:
df = sqlContext.read.load('s3://ui-hfpc/Performance_2015Q1.txt',
                          format='com.databricks.spark.csv',
                          header='false',
                          inferSchema='true',
                          delimiter='|',
                          nullValue=''
                          )

Note that on the `nullValue=''` line, the empty string can be replaced by whatever your dataset uses.

First let's see how many rows the entire dataframe has:

In [4]:
df.count()

3526154

To explore missing data in pySpark, we need to make sure we're looking in a numerical column - the system does not insert `null` into a column that has a string datatype.  The general point of `null` is so the system knows to skip those rows when doing calculations down a column.  

For example, the mean of the series [3, 4, 2, null, 5] is: 

14 / 4 = 3.5 

not: 

14 / 5 = 2.8

In [5]:
df.dtypes

[('C0', 'bigint'),
 ('C1', 'string'),
 ('C2', 'string'),
 ('C3', 'double'),
 ('C4', 'double'),
 ('C5', 'int'),
 ('C6', 'int'),
 ('C7', 'int'),
 ('C8', 'string'),
 ('C9', 'int'),
 ('C10', 'string'),
 ('C11', 'string'),
 ('C12', 'int'),
 ('C13', 'string'),
 ('C14', 'string'),
 ('C15', 'string'),
 ('C16', 'string'),
 ('C17', 'string'),
 ('C18', 'string'),
 ('C19', 'string'),
 ('C20', 'string'),
 ('C21', 'string'),
 ('C22', 'string'),
 ('C23', 'string'),
 ('C24', 'string'),
 ('C25', 'string'),
 ('C26', 'int'),
 ('C27', 'string')]

For our practice purposes it doesn't matter what this data actually is, so we'll arbitrarily select a numerical column:

In [10]:
df.where( df['C12'].isNull() ).count()

3510294

This command takes the form `df.where(___).count()` where the blank is replaced with the desired condition.  In the code I used some extra spaces in between the brackets just to make this stand out - Python ignores extra horizonal space nestled in commands like that.

Note that if we left the `count()` method off the end then it would return an actual dataframe of all rows where column `C12` is `null`.  So if you wanted more than just the count you could explore that subset.

When we compare the null count to our earlier command, `df.count()`, we can see that column `C12` is mostly null values - there are 15,860 values in here, out of 3,526,154 rows.  A common need when exploring a dataset might be to check _all_ our numeric rows for null values.  However, the `isNull()` method can only be called on a column, not an entire dataframe, so I'll write a convenient Python function to do this for us:

In [17]:
def count_nulls(df):
    null_counts = []          #make an empty list to hold our results
    for col in df.dtypes:     #iterate through the column data types we saw above
        cname = col[0]
        ctype = col[1]
        if ctype != 'string': #calling isNull() on string columns will do the work but return 0, so we skip them for efficiency
            nulls = df.where( df[cname].isNull() ).count()
            result = tuple([cname, nulls])
            null_counts.append(result)
    return null_counts

null_counts = count_nulls(df)

In [18]:
null_counts

[('C0', 0),
 ('C3', 0),
 ('C4', 1945752),
 ('C5', 0),
 ('C6', 0),
 ('C7', 1),
 ('C9', 0),
 ('C12', 3510294),
 ('C26', 3526153)]

A quick note about Python programming in general, for those who may be new(er) to the language: one of the core precepts of Python is that code should be as easy to read as possible.  For the purpose of clairty I spread the code in that last function out vertically far more than was strictly necessary.  This bit of code would do the exact same thing:

In [18]:
"""
null_counts = []
for col in df.dtypes:
    if col[1] != 'string':
        null_counts.append(tuple([col[0], df.where(df[col[0]].isNull()).count()])))
""";

But despite accomplishing the same thing in 4 lines instead of 8, it violates the rules of Python style by looking like an unreadable jumble.  Much more on this can be found in the official Python PEP8 style guide, located at:

https://www.python.org/dev/peps/pep-0008/

If you'll be writing much Python code it's definitely worth looking over.

There are three things we can do with our `null` values now that we know what's in our dataframe.  We can ignore them, we can drop them, or we can replace them.  Remember, pySpark dataframes are immutable, so we can't actually change the original dataset.  All operations return an entirely new dataframe, though we can tell it to overwrite the existing one with `df = df.some_operation()` which ends up roughly equivalent.

In [19]:
df_drops = df.dropna(how='all', thresh=None, subset=['C4', 'C12', 'C26'])

In [20]:
df_drops.count()

1580403

The `df.dropna()` method can have three arguments, all of which are optional.  The `how` argument can be `any` or `all`; the first drops a row if _any_ value is `null`, the second drops a row only if _all_ values are.  

The `thresh` argument sets a threshold for the number of `null` entries in a row before it drops it.  It is set to an integer that specifies how many non-null arguments the row must have; if it has less than that figure it drops.

And finally the `subset` argument takes a list of columns that you want to look in for `null` values.  It does not actually subset the dataframe; it just checks in those three columns, then drops the row for the entire dataframe if that subset meets the criteria.

So we can see above that once we drop all rows that have `null` for columns `C4`, `C12` and `C26`, we're left with 1,580,403 rows.

In [21]:
df_fill = df.fillna(0, subset=['C12'])

The above line goes through all of column `C12` and replaces `null` values with a zero.  To verify we re-run the command on our new dataframe to count nulls that we used above:

In [22]:
df_fill.where( df_fill['C12'].isNull() ).count()

0

We see it replaced all 3,510,294 nulls we found earlier.  The first term in `fillna()` can be most any type and any value, and the subset list can be left off if the fill should be applied to all columns.  Note that `df.replace(a, b)` does this same thing, only you specify `a` as the value to be replaced and `b` as the replacement.  It also accepts the optional subset list.